In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")["train"].train_test_split(test_size=0.225, train_size=0.45)

In [2]:
dataset["test"].to_pandas()

,text,label
0,"After reading only two of the comments herein,...",1
1,Let me state at the outset that I have Cerebra...,1
2,I'm not tired to say this is one of the best p...,1
3,To be a Buster Keaton fan is to have your hear...,0
4,I don't usually write a comment when there are...,0
...,...,...
6245,Since Douglas MacArthur affected more human li...,1
6246,This is an important film. It challenges the v...,1
6247,"Jack Frost, no kids it's not the warm hearted ...",0
6248,I watched this movie with some friends a coupl...,0


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(data):
    return tokenizer(data["text"], padding=True, truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

In [4]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=750,
    logging_dir="./imdb_logs",
    logging_steps=10,
    save_steps=1000,
    output_dir="./imdb_results",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


PermissionError: [WinError 5] Access is denied: './imdb_results\\tmp-checkpoint-5' -> './imdb_results\\checkpoint-5'

In [7]:
model.save_pretrained("./imdb-sentiment")
tokenizer.save_pretrained("./imdb-sentiment_pos_neg")

('./sentiment_pos_neg/tokenizer_config.json',
 './sentiment_pos_neg/special_tokens_map.json',
 './sentiment_pos_neg/vocab.txt',
 './sentiment_pos_neg/added_tokens.json',
 './sentiment_pos_neg/tokenizer.json')